# Homework 4: Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

The _actual sequences_ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
    
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
    
The end of NA is:

    ...CACGATAGATAAATAATAGTGCACCAT
    
The end of HA is:

    ...CCGGATTTGCATATAATGATGCACCAT
    
The _sequencing reads_ (located in the FASTQ file) are from the reverse end of these actual sequences, so the first thing in the sequencing reads is the reverse complement of the barcode followed by the reverse complement of the constant sequence, etc.
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

In [1]:
#Import Packages
import re
import Bio.SeqIO

In [2]:
#Read in sequencing data(from Lecture 9)
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', format='fastq'))

In [3]:
#Convert SeqRecords objects to a list of strings(Lecture 9)
seqreads_str = []
for seqrecord in seqreads:
    sequence = str(seqrecord.seq) # convert sequence part to string
    seqreads_str.append(sequence) # add string sequence to list

In [50]:
#Converst list of sequences to dataframe 
import pandas as pd
import numpy as np

df = pd.DataFrame (seqreads_str, columns = ['Sequence'])


In [51]:
#Make a function to get R.C from each string(Lecture 9)
def reverse_complement(seq, unk_partner = 'N'):
    base_partner = {'A':'T', 'T':'A', 'C':'G', 'G':'C'} #Make Dictionary of basepairs
    rseq = '' #Make Empty String
    for a in seq: #loops over bases in sequence
        if a in base_partner:
            # look up the complementary base in the dictionary
            pair = base_partner[a]
            rseq = pair + rseq #build reverse comp string
        else:
            rseq = unk_partner + rseq
    return rseq 

In [54]:
#Add a column of the Reverse compliment for each sequence 
df['Reverse_Comp'] = df['Sequence'].apply(reverse_complement)
df

,Sequence,Reverse_Comp
0,GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATC...,TCTGCTATTCCGTGTCAATAAACATCGTCGTTGCATGCCTTCATTT...
1,CTTCCTGGTCACGGTTGCGGCCGCCTATGGTGCATCATTATATGCA...,TCGTACACTCTGTCATTAGGGATGTATTTGTTTAATGCATGGGGTT...
2,CACGGCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCT...,TCGTAGTGTATAGTAGAAGGGACGTCTACGTTAATCAGTGTCATAA...
3,TCAGCGATGAATGTGGGCGGCCGCCTATGTTGCATCATTATATGCA...,TCAGATGAATGGTAGTTGGTGATAGCATGAGGTTGGGTCGGATGGT...
4,ATATGGGAGACGATAAGCGGCCGCCTATGGTGCATCATTATATGCA...,GAGTAAAGACTGTGTTCTGGGACGCGATCGAGTCTGCGAATGTGTG...
...,...,...
9995,GCCGTATGGCTGGTGGGCGGCCGCCTATGGTGCACTATTATTTATC...,GGGTTGTAGGTGTCTTCTGAGTCACTGTCGAATCAGGAGAGGTGTG...
9996,GGACAGCAGGGAGCGGGCGGCCGCCTATGGTGCATCATTATATGCA...,GCGTATAATCTGTAACCTCAATTGCGGCTCAGTAGTGCCCGGGGTC...
9997,TTTATATGAGTTTCGTGCGGCCGCCTATGGTGCACTATTATTTATC...,CGGGAGTAGTGGAGTGTCGCAACGTGCTCGCTTATAGGATTCTGGC...
9998,GGAGTTTATAACATGCGCGGCCGCCTATGGTGCACTATTATTTATC...,TCGTAAGAACGGTGTTCAGGATTACGATGAATTAAAGCCTGTAGCC...


In [57]:
#Make Function to Determine if sequences are NA or HA, making new column in dataframe called Strain that will be populated
#with either NA, HA or none.
#NAend='CACGATAGATAAATAATAGTGCACCAT'
#HAend='CCGGATTTGCATATAATGATGCACCAT'

def read_type(seqreads):
    
    # compile the type search pattern
    type_pattern1 = re.compile("(CACGATAGATAAATAATAGTGCACCAT)") 
    match1 = type_pattern1.search(seqreads)

    type_pattern2 = re.compile("(CCGGATTTGCATATAATGATGCACCAT)") 
    match2 = type_pattern2.search(seqreads) 
    if match1:
        return "NA"
    elif match2:
        return "HA"
    else:
        return None

    

In [58]:
#Add a column to evalulate if the Sequence is the NA or HA virus strain
df['Strain'] = df['Reverse_Comp'].apply(read_type)
df



,Sequence,Reverse_Comp,Strain
0,GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATC...,TCTGCTATTCCGTGTCAATAAACATCGTCGTTGCATGCCTTCATTT...,NA
1,CTTCCTGGTCACGGTTGCGGCCGCCTATGGTGCATCATTATATGCA...,TCGTACACTCTGTCATTAGGGATGTATTTGTTTAATGCATGGGGTT...,HA
2,CACGGCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCT...,TCGTAGTGTATAGTAGAAGGGACGTCTACGTTAATCAGTGTCATAA...,NA
3,TCAGCGATGAATGTGGGCGGCCGCCTATGTTGCATCATTATATGCA...,TCAGATGAATGGTAGTTGGTGATAGCATGAGGTTGGGTCGGATGGT...,None
4,ATATGGGAGACGATAAGCGGCCGCCTATGGTGCATCATTATATGCA...,GAGTAAAGACTGTGTTCTGGGACGCGATCGAGTCTGCGAATGTGTG...,HA
...,...,...,...
9995,GCCGTATGGCTGGTGGGCGGCCGCCTATGGTGCACTATTATTTATC...,GGGTTGTAGGTGTCTTCTGAGTCACTGTCGAATCAGGAGAGGTGTG...,NA
9996,GGACAGCAGGGAGCGGGCGGCCGCCTATGGTGCATCATTATATGCA...,GCGTATAATCTGTAACCTCAATTGCGGCTCAGTAGTGCCCGGGGTC...,HA
9997,TTTATATGAGTTTCGTGCGGCCGCCTATGGTGCACTATTATTTATC...,CGGGAGTAGTGGAGTGTCGCAACGTGCTCGCTTATAGGATTCTGGC...,NA
9998,GGAGTTTATAACATGCGCGGCCGCCTATGGTGCACTATTATTTATC...,TCGTAAGAACGGTGTTCAGGATTACGATGAATTAAAGCCTGTAGCC...,NA


In [59]:
#Make function to read barcodes(lecture 9)
def read_barcode(seqread, bclen=16, upstream='AGGCGGCCGC'):
    # compile the barcode search pattern
    barcode_pattern = re.compile(upstream + f"(?P<barcode>[ATCGN]{{{bclen}}})$")
    
    # search for the barcode pattern
    match = barcode_pattern.search(seqread)
    
    if match:
        barcode = match.group('barcode')
        return barcode
    else:
        return None

In [60]:
#Add barcode column to dataframe
df['Barcode'] = df['Reverse_Comp'].apply(read_barcode)
df

,Sequence,Reverse_Comp,Strain,Barcode
0,GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATC...,TCTGCTATTCCGTGTCAATAAACATCGTCGTTGCATGCCTTCATTT...,NA,ACTAAATAACTTAAGC
1,CTTCCTGGTCACGGTTGCGGCCGCCTATGGTGCATCATTATATGCA...,TCGTACACTCTGTCATTAGGGATGTATTTGTTTAATGCATGGGGTT...,HA,AACCGTGACCAGGAAG
2,CACGGCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCT...,TCGTAGTGTATAGTAGAAGGGACGTCTACGTTAATCAGTGTCATAA...,NA,None
3,TCAGCGATGAATGTGGGCGGCCGCCTATGTTGCATCATTATATGCA...,TCAGATGAATGGTAGTTGGTGATAGCATGAGGTTGGGTCGGATGGT...,None,CCACATTCATCGCTGA
4,ATATGGGAGACGATAAGCGGCCGCCTATGGTGCATCATTATATGCA...,GAGTAAAGACTGTGTTCTGGGACGCGATCGAGTCTGCGAATGTGTG...,HA,TTATCGTCTCCCATAT
...,...,...,...,...
9995,GCCGTATGGCTGGTGGGCGGCCGCCTATGGTGCACTATTATTTATC...,GGGTTGTAGGTGTCTTCTGAGTCACTGTCGAATCAGGAGAGGTGTG...,NA,CCACCAGCCATACGGC
9996,GGACAGCAGGGAGCGGGCGGCCGCCTATGGTGCATCATTATATGCA...,GCGTATAATCTGTAACCTCAATTGCGGCTCAGTAGTGCCCGGGGTC...,HA,CCGCTCCCTGCTGTCC
9997,TTTATATGAGTTTCGTGCGGCCGCCTATGGTGCACTATTATTTATC...,CGGGAGTAGTGGAGTGTCGCAACGTGCTCGCTTATAGGATTCTGGC...,NA,ACGAAACTCATATAAA
9998,GGAGTTTATAACATGCGCGGCCGCCTATGGTGCACTATTATTTATC...,TCGTAAGAACGGTGTTCAGGATTACGATGAATTAAAGCCTGTAGCC...,NA,GCATGTTATAAACTCC


In [74]:
#Count how many of each strain there is
df.groupby("Strain")["Strain"].count() 


Strain
HA    5409
NA    4122
Name: Strain, dtype: int64

1. How many reads map to HA, and how many reads map to NA?
41.22% (4122 reads) of the sequences are NA strain and 54.09%(5409 reads) are HA strain with 4.69%(469 reads) being neither NA or HA.

In [89]:
#Getting barcode counts for each strain
just_NA = df.loc[df['Strain'] == 'NA'] 

just_HA = df.loc[df['Strain'] == 'HA']


print(just_NA['Barcode'].value_counts())

print(just_HA['Barcode'].value_counts())


ACCAGTTCTCCCCGGG    152
TCAAGAAGCCTTGGAG    150
TGACGATCCTCAAGAA    141
CGTCTTCCATCCCCAT    133
AATCGTACATCCCCAC    123
                   ... 
TGTACAGCAAGAGTAG      1
GTTCAGGATACACTTA      1
TCAAGAAGCCTTGGTG      1
GAACCCTAGAAGCCCC      1
GCACATAAGGAGCCAA      1
Name: Barcode, Length: 138, dtype: int64
CCCGACCCGACATTAA    155
GATTTCCGATCAGTCT    124
GCTACTACTATACCAT    119
CTCCATCACTCGCCAA    115
AACCGTCACCTCAACC    104
                   ... 
CATCACGCCCCTTAGC      1
TGAATTACCTAACCCG      1
CGGCCCCAACGCAATG      1
AGAGGCGGAAGGCGTC      1
GCTACTACTATACCTT      1
Name: Barcode, Length: 281, dtype: int64


2. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

For the NA strain the most common barode is ACCAGTTCTCCCCGGG occuring 152 times.
For the HA strain the most common barcode is CCCGACCCGACATTAA occuring 155 times.
